# Extração dos pesos e bias de uma rede neural no python

Para a construção da rede neural no python, considerou-se os seguintes aspectos:

- Utilização do framework Tensorflow para criação e treinamento da rede neural;
- Criação de uma rede neural com 4 classes (saudável, BPFI, BPFO e misto) a partir dos seguintes arquivos:
    - feature_healthy_vibration_1s.csv
    - BPFO_1mm_vibration_1s.csv
    - BPFI_05mm_vibration_1s.csv
    - Mixed_failure_vibration_1s.csv
- Divisão dos dados em 70% treinamento, 15% validação e 15% teste
- Função de ativação ReLU nas camadas intermediárias e Softmax na camada de saída
- Sem batch normalization
- Remoção dos parâmetros sd (standard deviation) e RMS.

## Leitura dos dados

In [5]:
import pandas as pd 

saudavel = pd.read_csv('/home/thyago/Documentos/GitHub/mit-ufpb-analysis/Planilhas/Rolamento_saudavel_6206ZZ/feature_healthy_vibration_1s.csv')
bpfi = pd.read_csv('/home/thyago/Documentos/GitHub/mit-ufpb-analysis/Planilhas/Rolamento_falha_BPFI_05mm/BPFI_05mm_vibration_1s.csv')
bpfo = pd.read_csv('/home/thyago/Documentos/GitHub/mit-ufpb-analysis/Planilhas/Rolamento_falha_BPFO_1mm/BPFO_1mm_vibration_1s.csv')
misto = pd.read_csv('/home/thyago/Documentos/GitHub/mit-ufpb-analysis/Planilhas/Falha_mista/Mixed_failure_vibration_1s.csv')

df = pd.concat([saudavel, bpfi, bpfo, misto], ignore_index=True)

In [7]:
df.head()

,max,min,mean,sd,rms,skewness,kurtosis,crest,form,fault
0,4.961,-4.961,0.001054,1.307086,1.307066,-0.029449,2.672257,3.795524,1.228393,Normal
1,4.348,-4.348,0.007885,1.319856,1.319858,0.012514,2.626463,3.294293,1.229056,Normal
2,4.459,-4.186,-0.007867,1.397529,1.397529,0.001150,2.577867,3.190632,1.226880,Normal
3,4.260,-4.260,-0.015230,1.369683,1.369746,-0.016739,2.639587,3.110065,1.235105,Normal
4,4.582,-4.582,-0.011721,1.352148,1.352178,-0.038328,2.607397,3.388608,1.227075,Normal


## Remoção dos parâmetros sd e RMS

In [9]:
df = df.drop(columns=['sd', 'rms'])

df.head()

,max,min,mean,skewness,kurtosis,crest,form,fault
0,4.961,-4.961,0.001054,-0.029449,2.672257,3.795524,1.228393,Normal
1,4.348,-4.348,0.007885,0.012514,2.626463,3.294293,1.229056,Normal
2,4.459,-4.186,-0.007867,0.001150,2.577867,3.190632,1.226880,Normal
3,4.260,-4.260,-0.015230,-0.016739,2.639587,3.110065,1.235105,Normal
4,4.582,-4.582,-0.011721,-0.038328,2.607397,3.388608,1.227075,Normal


## Divisão dos dados em treinamento e teste

In [17]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

# Conversão da coluna 'fault' para valores numéricos utilizando LabelEncoder
le = LabelEncoder()
df['fault'] = le.fit_transform(df['fault'])

# Separação das variáveis de entrada e saída
X = df.drop(columns=['fault'])
y = df['fault']

# Divisão dos dados em treinamento (70%), teste (15%) e validação (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3)          # 70% treinamento e 30% teste
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5)    # Dos 30% restantes, 50% teste e 50% validação
 
# Imprima os tamanhos dos conjuntos
print(f'Treinamento: {len(X_train)} amostras')
print(f'Teste: {len(X_test)} amostras')
print(f'Validação: {len(X_val)} amostras')

Treinamento: 3427 amostras
Teste: 734 amostras
Validação: 735 amostras


## Treinamento da Rede Neural

In [37]:
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import tensorflow as tf

# Normalização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Converta as labels para one-hot encoding
y_train_one_hot = pd.get_dummies(y_train).values
y_val_one_hot = pd.get_dummies(y_val).values

# criando um modelo de rede neural sequencial
model = Sequential()

# Adicionando camadas ao modelo
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))

# Compile o modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq=1)

# Treine o modelo usando dados de treinamento e valide usando dados de validação
model.fit(X_train_scaled, y_train_one_hot, epochs=50, batch_size=8, validation_data=(X_val_scaled, y_val_one_hot))

# Avalie o modelo no conjunto de teste
X_test_scaled = scaler.transform(X_test)
y_pred_test = model.predict(X_test_scaled)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)
accuracy_test = accuracy_score(y_test, y_pred_test_classes)
print(f'Acurácia no conjunto de teste: {accuracy_test}')


Epoch 1/50
429/429 [==============================] - 4s 7ms/step - loss: 0.4731 - accuracy: 0.8564 - val_loss: 0.2434 - val_accuracy: 0.9184
Epoch 2/50
429/429 [==============================] - 2s 5ms/step - loss: 0.1550 - accuracy: 0.9484 - val_loss: 0.1607 - val_accuracy: 0.9293
Epoch 3/50
429/429 [==============================] - 2s 4ms/step - loss: 0.1034 - accuracy: 0.9597 - val_loss: 0.0949 - val_accuracy: 0.9701
Epoch 4/50
429/429 [==============================] - 2s 5ms/step - loss: 0.0890 - accuracy: 0.9670 - val_loss: 0.0936 - val_accuracy: 0.9633
Epoch 5/50
429/429 [==============================] - 2s 4ms/step - loss: 0.0819 - accuracy: 0.9694 - val_loss: 0.0927 - val_accuracy: 0.9646
Epoch 6/50
429/429 [==============================] - 2s 4ms/step - loss: 0.0759 - accuracy: 0.9723 - val_loss: 0.0895 - val_accuracy: 0.9633
Epoch 7/50
429/429 [==============================] - 2s 4ms/step - loss: 0.0739 - accuracy: 0.9723 - val_loss: 0.0787 - val_accuracy: 0.9687
Epoch 

In [39]:
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt

# Representação gráfica do modelo
plot_model(model, to_file='imagens/model-1.png', show_shapes=True, show_layer_names=True)
from tensorflow.keras.models import save_model

save_model(model, 'modelos/modelo.h5')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


## Acessando os pesos e biases ótimos do modelo treinado

In [40]:
# Acesse os pesos e bias
weights_biases_data = []

for layer in model.layers:
    if hasattr(layer, 'kernel'):
        weights_biases_data.append(f'Pesos ({layer.name}): {layer.get_weights()[0]}')
    if hasattr(layer, 'bias'):
        weights_biases_data.append(f'Bias ({layer.name}): {layer.get_weights()[1]}')

# Escreva os dados em um arquivo txt
with open('pesos_biases.txt', 'w') as file:
    for line in weights_biases_data:
        file.write(line + '\n')
